### Project 1: Using momentum to make the strategy

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
start = '2017-01-01'

In [3]:
overall = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

In [4]:
overall

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


### Data Handling
We will first preapre the data and clean the data to make sure the data is clean and reliable.

#### Problem: 

We are analysing the list of stocks from past 12 months for S&P 500, but in an index based on the performance in the market. Stocks are added and getst removed as well. This could lead to Survivors Bias. Survivor bias is when we concentrate on the values that passed the selection while overlooking that did not. 

#### Solution
We are going to analyze the data by taking the stocks that were removed, before the day they were removed. And exclude the stocks before they were added in the index

In [5]:
stocks = overall.Symbol

In [6]:
stocks.head(5)

0     MMM
1     AOS
2     ABT
3    ABBV
4     ACN
Name: Symbol, dtype: object

In [7]:
stocks = stocks.to_list()

In [8]:
#only those are relevant after the starting date of the analysis
overall = overall[overall['Date added'] >= start]


In [9]:
removed = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[1][['Date','Removed']]

In [10]:
removed

Date Removed                 
                  Date  Ticker         Security
0       March 24, 2025     BWA       BorgWarner
1       March 24, 2025     TFX         Teleflex
2       March 24, 2025      CE         Celanese
3       March 24, 2025     FMC  FMC Corporation
4    December 23, 2024    QRVO            Qorvo
..                 ...     ...              ...
363     April 12, 1999     NaN              NaN
364  December 11, 1998      AN            Amoco
365  December 11, 1998     GRN       General Re
366  December 11, 1998     SUN       SunAmerica
367      June 17, 1997     USL           USLife

[368 rows x 3 columns]

In [11]:
# Data manipulation
# Exatracting date from datetime64 column
removed = removed.set_index(removed.Date.Date)

In [12]:
# making sure the date column is a datetime64 type
removed.index = pd.to_datetime(removed.index)

In [13]:
# filtering the data to only include dates at and after the start date
removed = removed[removed.index >= start]

In [14]:
# dropping any rows with NaN values in the 'Removed' column
removed = removed.Removed.dropna()

In [15]:
removed

,Ticker,Security
Date,,
2025-03-24,BWA,BorgWarner
2025-03-24,TFX,Teleflex
2025-03-24,CE,Celanese
2025-03-24,FMC,FMC Corporation
2024-12-23,QRVO,Qorvo
...,...,...
2017-03-13,LLTC,Linear Technology
2017-03-02,ENDP,Endo International
2017-03-01,PBI,Pitney Bowes


In [16]:
# include already removed symbols in our analysis
stocks.extend(removed.Ticker.to_list())


In [17]:
# include already removed symbols in our analysis
df = yf.download(stocks,start=start, auto_adjust=False)['Close']

[*********************100%***********************]  661 of 661 completed

72 Failed downloads:
['XLNX', 'MXIM', 'CHK', 'CERN', 'DISCK', 'TIF', 'HFC', 'VIAB', 'CTLT', 'TWTR', 'FRC', 'PXD', 'DRE', 'RHT', 'XEC', 'LLL', 'ENDP', 'KSU', 'BRK.B', 'WCG', 'NLSN', 'VAR', 'CTXS', 'ADS', 'PBCT', 'CELG', 'SIVB', 'BBBY', 'ATVI', 'APC', 'NBL', 'YHOO', 'MNK', 'FLIR', 'ARNC', 'AGN', 'GPS', 'TSS', 'ETFC', 'SWN', 'MON', 'XL', 'RTN', 'DISCA', 'FBHS', 'ALXN', 'CXO', 'FTR', 'MRO', 'DWDP', 'ABMD', 'DISH']: YFTzMissingError('possibly delisted; no timezone found')
['LLTC', 'RAI', 'WFM', 'WYN', 'LEN', 'GGP', 'SPLS', 'MJN', 'SNI', 'BF.B', 'STJ', 'BCR', 'DPS', 'LVLT']: YFPricesMissingError('possibly delisted; no price data found  (1d 2017-01-01 -> 2025-04-08)')
['AET', 'TWX', 'ANDV', 'SCG', 'CSRA', 'BMS']: ValueError('Length mismatch: Expected axis has 2 elements, new values have 1 elements')


As we can see from above observed DF, we have the info till the present day however we will only keep the info till the stock got removed from the index.

In [18]:
ticker = 'BWA'  # Example ticker to remove
# Check if the ticker exists in the DataFrame
if ticker in df.columns:
    removal_date = removed[removed.Ticker == ticker].index[0]
    print(f"Removing data for {ticker} after: {removal_date}")

    df.loc[df.index > removal_date, ticker] = np.nan

    # print("First 5 values after cleaning:")
    print(df[ticker].dropna().head())
else:
    print(f"{ticker} not found in df.columns")

Removing data for BWA after: 2025-03-24 00:00:00
Date
2017-01-03    35.272888
2017-01-04    35.933098
2017-01-05    35.889084
2017-01-06    36.179577
2017-01-09    35.933098
Name: BWA, dtype: float64


In [19]:
df['BWA']

Date
2017-01-03    35.272888
2017-01-04    35.933098
2017-01-05    35.889084
2017-01-06    36.179577
2017-01-09    35.933098
                ...    
2025-04-01          NaN
2025-04-02          NaN
2025-04-03          NaN
2025-04-04          NaN
2025-04-07          NaN
Name: BWA, Length: 2077, dtype: float64

As we can see no price after 2025-03-24 for BWA, Borg Warner.

Next, we will define functions for the simplification of removal and exclusion of the ticker symbol.

In [20]:
# Function to remove data for a specific ticker after a certain date
def pricefilter_rem(ticker):
    df[ticker] = df[ticker][df[ticker].index <= removed[removed.Ticker ==ticker].index[0] ]

# Function for the tickers that were added
def pricefilter_add(ticker):
    df[ticker] = df[ticker][df[ticker].index >= overall[overall.Symbol ==ticker]['Date added'].values[0]]

In [21]:
# Usinng for loop to apply the function to all tickers
for ticker_rem in removed.Ticker:
    pricefilter_rem(ticker_rem)
for ticker_add in overall.Symbol:
    pricefilter_add(ticker_add)

In [22]:
removed

,Ticker,Security
Date,,
2025-03-24,BWA,BorgWarner
2025-03-24,TFX,Teleflex
2025-03-24,CE,Celanese
2025-03-24,FMC,FMC Corporation
2024-12-23,QRVO,Qorvo
...,...,...
2017-03-13,LLTC,Linear Technology
2017-03-02,ENDP,Endo International
2017-03-01,PBI,Pitney Bowes


We can see the BWA was removed on 2025-03-04. Let's do a proof of check to see if the functions are working correctly.

In [23]:
# to check if the data is correct
df['BWA'].tail(10)

Date
2025-03-25   NaN
2025-03-26   NaN
2025-03-27   NaN
2025-03-28   NaN
2025-03-31   NaN
2025-04-01   NaN
2025-04-02   NaN
2025-04-03   NaN
2025-04-04   NaN
2025-04-07   NaN
Name: BWA, dtype: float64

In [24]:
df['PBI'].head(45)

Date
2017-01-03    15.860000
2017-01-04    15.930000
2017-01-05    15.970000
2017-01-06    16.180000
2017-01-09    16.100000
2017-01-10    16.200001
2017-01-11    16.080000
2017-01-12    16.299999
2017-01-13    16.590000
2017-01-17    16.219999
2017-01-18    16.209999
2017-01-19    15.670000
2017-01-20    15.720000
2017-01-23    15.700000
2017-01-24    16.049999
2017-01-25    16.180000
2017-01-26    15.850000
2017-01-27    15.870000
2017-01-30    15.700000
2017-01-31    15.920000
2017-02-01    13.140000
2017-02-02    12.940000
2017-02-03    13.180000
2017-02-06    13.160000
2017-02-07    13.070000
2017-02-08    13.080000
2017-02-09    13.180000
2017-02-10    13.190000
2017-02-13    13.210000
2017-02-14    13.490000
2017-02-15    13.350000
2017-02-16    13.280000
2017-02-17    13.390000
2017-02-21    13.760000
2017-02-22    13.830000
2017-02-23    13.810000
2017-02-24    14.070000
2017-02-27    13.990000
2017-02-28    13.640000
2017-03-01    13.800000
2017-03-02          NaN
2017-03-03 

In [25]:
# first_nan_date = df['BWA'][df['BWA'].isna()].index[0]
# print(f"First NaN date for PBI: {first_nan_date}")

We can see both BWA and PBI's exclusion date.

Till now we added stocks cleaned the data for analysing.
Now we will move to the second part of the project.

### Part 2: 
Now we will calculate the returns. Benchamrk and compare the results.

In [26]:
ret_df = df.pct_change()

C:\Users\aditya\AppData\Local\Temp\ipykernel_6464\902390033.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret_df = df.pct_change()


In [27]:
ret_df

Ticker,A,AAL,AAP,AAPL,ABBV,ABMD,ABNB,ABT,ACGL,ACN,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-04,0.013121,0.008639,0.008206,-0.001119,0.014100,NaN,NaN,0.007939,NaN,0.002404,...,-0.011002,0.008188,0.037736,0.014904,NaN,0.003639,0.009194,NaN,0.014358,0.009703
2017-01-05,-0.011890,-0.017345,-0.000698,0.005085,0.007584,NaN,NaN,0.008638,NaN,-0.014991,...,-0.014907,-0.013029,-0.004196,-0.009129,NaN,0.003310,0.006425,NaN,-0.016210,-0.003327
2017-01-06,0.031156,0.006973,-0.013091,0.011148,0.000314,NaN,NaN,0.027204,NaN,0.011392,...,-0.000565,-0.000686,-0.015449,-0.007010,NaN,0.012097,0.000095,NaN,0.006498,0.003152
2017-01-09,0.003126,0.018827,-0.000590,0.009159,0.006584,NaN,NaN,-0.000981,NaN,-0.011178,...,-0.016497,0.003088,-0.005706,-0.004236,NaN,0.002794,0.019436,NaN,-0.010837,-0.002773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-01,-0.025047,0.000000,0.000000,0.004772,-0.015512,NaN,0.014398,-0.007237,-0.000416,0.004935,...,0.000925,0.000000,0.000000,0.003516,NaN,0.009914,-0.007157,0.001805,0.000000,-0.016641
2025-04-02,0.017010,0.000000,0.000000,0.003136,-0.005236,NaN,0.013369,0.001822,0.007073,0.008770,...,-0.003108,0.000000,0.000000,-0.006924,NaN,-0.004342,0.001157,0.015897,0.000000,0.009820
2025-04-03,-0.054487,0.000000,0.000000,-0.092456,-0.017301,NaN,-0.071906,-0.002274,-0.005577,-0.047008,...,-0.052583,0.000000,0.000000,-0.067703,NaN,0.020350,0.001422,-0.170637,0.000000,-0.023792


In [28]:
# Calculating the returns for one month
monthly_ret = (ret_df+1).resample('ME').prod()

In [29]:
monthly_ret['AAPL'].head(10)

Date
2017-01-31    1.044770
2017-02-28    1.128883
2017-03-31    1.048690
2017-04-30    0.999930
2017-05-31    1.063418
2017-06-30    0.942786
2017-07-31    1.032704
2017-08-31    1.102669
2017-09-30    0.939756
2017-10-31    1.096808
Freq: ME, Name: AAPL, dtype: float64

In [30]:
# Returns for the past 12 months
returns_12month = monthly_ret.rolling(12).apply(np.prod).dropna()

In [31]:
returns_12month

Ticker,A,AAL,AAP,AAPL,ABBV,ABMD,ABNB,ABT,ACGL,ACN,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2017-12-31,1.440525,1.123758,0.584349,1.456995,1.549591,1.0,1.000000,1.461460,1.000000,1.314529,...,0.920233,1.122996,1.057692,1.373615,1.0,1.291093,1.167812,1.000000,1.177165,1.344281
2018-01-31,1.499489,1.227571,0.712311,1.379728,1.836361,1.0,1.000000,1.488149,1.000000,1.411258,...,1.040649,1.072487,1.231241,1.465423,1.0,1.290859,1.074284,1.000000,1.280635,1.396615
2018-02-28,1.337037,1.170190,0.729519,1.300241,1.873060,1.0,1.000000,1.338287,1.000000,1.314367,...,0.931382,0.882557,1.018817,1.549875,1.0,1.245866,0.992911,1.000000,1.224276,1.516789
2018-03-31,1.265368,1.228369,0.799609,1.167896,1.452578,1.0,1.000000,1.349246,1.000000,1.280447,...,0.909767,0.805734,0.980245,1.531661,1.0,1.332238,0.892965,1.000000,1.255476,1.564737
2018-04-30,1.194187,1.007274,0.805192,1.150435,1.464210,1.0,1.000000,1.332035,1.000000,1.246496,...,0.952235,0.796015,1.093533,1.418012,1.0,1.324715,0.962557,1.000000,1.367724,1.487792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,0.966266,0.799127,1.000000,1.300680,1.146674,1.0,0.965256,1.027619,1.243436,1.002508,...,1.075915,0.894914,1.000000,1.014516,1.0,1.026787,0.867954,1.413017,0.905630,0.825505
2025-01-31,1.164643,0.771609,1.000000,1.279827,1.118613,1.0,0.910018,1.130623,1.129079,1.057904,...,1.039101,0.916547,1.000000,1.103166,1.0,1.007800,0.871656,1.636151,0.948210,0.909962
2025-02-28,0.931275,0.700255,1.000000,1.337981,1.187333,1.0,0.881882,1.163267,1.060738,0.929879,...,1.065155,0.974602,1.000000,1.030224,1.0,1.129678,0.838855,1.127272,1.007608,0.843241


In [32]:
# Now for the 1 month retun since we are holding stocks for 1 month
# We are looking at the highest retunrs from top 5 stocks
top_ = returns_12month.loc['2017-12-31'].nlargest(5)

In [33]:
top_

Ticker
NKTR    4.706068
SEDG    2.844697
NRG     2.315447
PENN    2.221986
IPGP    2.167747
Name: 2017-12-31 00:00:00, dtype: float64

The result above is calculation done easier to compute. In simple terms SEDG which is Solaredge Technologies Inc would have given a stock incxrease of $2.88 in 12 months.

Next, we are extracting the top performance of the top performers *top_*

In [34]:
top_.name

Timestamp('2017-12-31 00:00:00')

In [35]:
returns_12month[top_.name:][1:2]

Ticker,A,AAL,AAP,AAPL,ABBV,ABMD,ABNB,ABT,ACGL,ACN,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,1.499489,1.227571,0.712311,1.379728,1.836361,1.0,1.0,1.488149,1.0,1.411258,...,1.040649,1.072487,1.231241,1.465423,1.0,1.290859,1.074284,1.0,1.280635,1.396615


In [36]:
actual_ret_monthly = returns_12month[top_.name:][1:2][top_.index]
actual_ret_monthly

Ticker,NKTR,SEDG,NRG,PENN,IPGP
Date,,,,,
2018-01-31,6.904212,2.772201,1.572551,2.315675,2.19106


Let's say we bought the top 5 stocks from the observed data, we can calculate the returns. Assuming the portfolio is equally divided.

In [47]:
# mean return
actual_ret_monthly.mean(axis=1)

Date
2018-01-31    3.15114
Freq: ME, dtype: float64

Our mean return would be 31%.

Next, we will create a function that looks at top 5 performers for a particular month. Then, look at the returns of those 5 stocks in the following month and return their average return.

In [55]:
# Creating a function to get the top 5 performers for a given date
def top_performers(date):
    all = returns_12month.loc[date]
    top = all.nlargest(5)
    actual_ret_monthly = returns_12month[top.name:][1:2][top.index]
    return (actual_ret_monthly).mean(axis=1).values[0]

In [57]:
# Example usage of the function
# function shows how much the top 5 stocks have returned in the listed month
top_performers('2018-01-31')

3.266843731360084

In [61]:
returns = []
for date in returns_12month.index[:-1]:
    returns.append(top_performers(date))

We skipped the last element, because we cannot know the last return [:-1].

In [ ]:
# Accumulating the returns
pd.Series(returns).prod()

2.9344442310300275e+30

### Benchmark comparison, by pulling the data for S&P 500 only.

In [69]:
s_p500 = yf.download('^GSPC', start=start).Close

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [70]:
s_p500

Ticker,^GSPC
Date,
2017-01-03,2257.830078
2017-01-04,2270.750000
2017-01-05,2269.000000
2017-01-06,2276.979980
2017-01-09,2268.899902
...,...
2025-04-01,5633.069824
2025-04-02,5670.970215
2025-04-03,5396.520020


In [71]:
2257.830078/5062.250000

0.4460131518593511

We outperformed the S&P 500 by 2.25 times, without the bias.

* In this project we used Cross-functional momentum startegy to compare different stocks and what would be their relative retun.